## Information Retrieval

### Importing the required libraries.

In [1]:
from bs4 import BeautifulSoup
import requests
import nltk
import time
import schedule
from datetime import datetime
import re
import csv
import nltk 

In [3]:
url = "https://pureportal.coventry.ac.uk/robots.txt"
page = requests.get(url)
print(page.text)

User-Agent: *
Crawl-Delay: 5
Disallow: /*?*format=rss
Disallow: /*?*export=xls

User-agent: GPTBot
Disallow: /

User-agent: ChatGPT-User
Disallow: /

User-agent: Google-Extended
Disallow: /

Sitemap: https://pureportal.coventry.ac.uk/sitemap.xml


In [5]:
current_time = datetime.now()
def CGL_crawler(seed, maxcount):
    # Initialize the queue with the given seed URL
    Q = [seed]
    count = 0
    time.sleep(5)  # Add a 5-second time delay to obey the website's rate limits

    # Perform the crawling until the queue is empty or the maxcount is reached
    while (Q != [] and count < maxcount):
        count += 1
        # Get the next URL to fetch from the front of the queue
        url = Q.pop(0)
        print("Fetching " + url)

        # Send an HTTP request and fetch 
        code = requests.get(url)
        plain_html = code.text

        # Parse the HTML with help of BeautifulSoup
        s = BeautifulSoup(plain_html, "html.parser")

        # Extracting all anchor tags 
        for link in s.findAll('a'):
            new_url = link.get('href')
            if new_url is not None and new_url != '/':
                new_url = new_url.strip()

                # Normalize the URL link
                if new_url[0:7] != 'http://' and new_url[0:8] != 'https://':
                    if url[len(url) - 1] == '/':
                        new_url = url + new_url
                    else:
                        new_url = url + '/' + new_url

                # Add the new URL to preform crawling
                Q.append(new_url)

# the number of pages to crawl
CGL_crawler('https://pureportal.coventry.ac.uk/en/organisations/centre-global-learning/publications/', 5)

# current time
print("Current Time:", current_time)

# Schedule the crawler to run every 1 week
schedule.every(1).week.do(CGL_crawler, 'https://pureportal.coventry.ac.uk/en/organisations/centre-global-learning/publications/')

Fetching https://pureportal.coventry.ac.uk/en/organisations/centre-global-learning/publications/
Fetching https://pureportal.coventry.ac.uk/en/organisations/centre-global-learning/publications/#main-navigation
Fetching https://pureportal.coventry.ac.uk/en/organisations/centre-global-learning/publications/#main-search
Fetching https://pureportal.coventry.ac.uk/en/organisations/centre-global-learning/publications/#main-content
Fetching https://pureportal.coventry.ac.uk/en/organisations/centre-global-learning/publications//en/
Current Time: 2024-10-17 12:28:58.203505


Every 1 week do CGL_crawler('https://pureportal.coventry.ac.uk/en/organisations/centre-global-learning/publications/') (last run: [never], next run: 2024-10-24 12:29:10)

In [6]:
def CGL_crawler(seed, maxcount, num_pages):
# create a dictnonary and list of variables 
    website_data = {
        "Dates": [],
        "Title": [],
        "Publication_urls": [],
        "Author": [],
        "Author_Link": [],
        "Publication_authors": []  # Store a list of authors for each publication
    }
    # For loop to go through pages
    for page in range(0,num_pages+1):
        url = seed + f'page={page}'
        Q = [(url, 1)] 
        count = 0
        time.sleep(5)  # Add a 5-second time delay to obey the website's rate limits
        while Q != [] and count < maxcount:
            count += 1
            url, num_pages = Q.pop(0)
            # This is to print the pages we wanted to crawl
            print("Fetching " + url)
            code = requests.get(url)
            # TO get all the html code in the page
            plain = code.text
            soup = BeautifulSoup(plain, "html.parser")
            # Append the Publication date
            dates = soup.find_all('span', class_='date')
            for date in dates:
                website_data["Dates"].append(date.text)
            # Append the Publication Title
            headings = soup.find_all('h3')
            for heading in headings:
                website_data["Title"].append(heading.text)
            # nested loop to find the Publication
            uls = soup.find_all('ul', class_='list-results')
            for ul in uls:
                links = ul.find_all('a')
                if ul.find_previous_sibling('h3'):
                    current_publication_text = ul.find_previous_sibling('h3').text
                    publication_url = ul.find_previous_sibling('h3').find('a')['href']
                else:
                    current_publication_text = ''
                    publication_url = ''

                for link in links:
                    link_text = link.text
                    link_url = link['href']
                    if link_url is not None and link_text != '' and link_url != '/':
                        if link_url.startswith('https://pureportal.coventry.ac.uk/en/publications'):
                            # Append the Publication URL 
                            website_data["Publication_urls"].append(link_url)
                            current_publication_text = link_text
                            website_data["Publication_authors"].append([]) 
                        elif link_url.startswith('https://pureportal.coventry.ac.uk/en/persons') and current_publication_text != '':
                            Author = link_text
                            Author_Link = link_url
                            # Append the author's name and URL to the corresponding lists
                            website_data["Author"].append(Author)
                            website_data["Author_Link"].append(Author_Link)
                            # Append the author's name and URL to the list of authors for the current publication
                            website_data["Publication_authors"][-1].append({"name": Author, "url": Author_Link})

    return website_data

website_data = CGL_crawler('https://pureportal.coventry.ac.uk/en/organisations/ics-research-centre-for-fluid-and-complex-systems-fcs/publications/?', 10, 9)
# Schedule the crawler to run every 1 week
# schedule.every(1).week.do(CGL_crawler, 'https://pureportal.coventry.ac.uk/en/organisations/centre-global-learning/publications/')
# Display all the information
for date, title, publication_url, publication_authors in zip(
    website_data["Dates"], website_data["Title"],
    website_data["Publication_urls"], website_data["Publication_authors"]
):
    print("Date:", date)
    print("Title:", title)
    print("Publication URL:", publication_url)
    print("Publication Authors:")
    for author in publication_authors:
        print("- Name:", author["name"])
        print("  URL:", author["url"])

Fetching https://pureportal.coventry.ac.uk/en/organisations/ics-research-centre-for-fluid-and-complex-systems-fcs/publications/?page=0
Fetching https://pureportal.coventry.ac.uk/en/organisations/ics-research-centre-for-fluid-and-complex-systems-fcs/publications/?page=1
Fetching https://pureportal.coventry.ac.uk/en/organisations/ics-research-centre-for-fluid-and-complex-systems-fcs/publications/?page=2
Fetching https://pureportal.coventry.ac.uk/en/organisations/ics-research-centre-for-fluid-and-complex-systems-fcs/publications/?page=3
Fetching https://pureportal.coventry.ac.uk/en/organisations/ics-research-centre-for-fluid-and-complex-systems-fcs/publications/?page=4
Fetching https://pureportal.coventry.ac.uk/en/organisations/ics-research-centre-for-fluid-and-complex-systems-fcs/publications/?page=5
Fetching https://pureportal.coventry.ac.uk/en/organisations/ics-research-centre-for-fluid-and-complex-systems-fcs/publications/?page=6
Fetching https://pureportal.coventry.ac.uk/en/organisat

In [7]:
# Display all the information
for date, title, publication_url, publication_authors in zip(
    website_data["Dates"], website_data["Title"],
    website_data["Publication_urls"], website_data["Publication_authors"]
):
    print("Date:", date)
    print("Title:", title)
    print("Publication URL:", publication_url)
    print("Publication Authors:")
    for author in publication_authors:
        print("- Name:", author["name"])
        print("  URL:", author["url"])

Date: 23 Apr 2024
Title: Aging following a zero-temperature quench in the d = 3 Ising model
Publication URL: https://pureportal.coventry.ac.uk/en/publications/aging-following-a-zero-temperature-quench-in-the-idi-3-ising-mode
Publication Authors:
- Name: Gessert, D.
  URL: https://pureportal.coventry.ac.uk/en/persons/denis-gessert
Date: 5 Jun 2024
Title: A New Phase Diagram for Fluid Invasion Patterns as a Function of Pore‐Scale Heterogeneity, Surface Roughness, and Wettability
Publication URL: https://pureportal.coventry.ac.uk/en/publications/a-new-phase-diagram-for-fluid-invasion-patterns-as-a-function-of-
Publication Authors:
- Name: Holtzman, R.
  URL: https://pureportal.coventry.ac.uk/en/persons/ran-holtzman
Date: 22 Mar 2024
Title: A Novel Cost-Effective Approach To Flow Visualization Using Particle Streak Angles
Publication URL: https://pureportal.coventry.ac.uk/en/publications/a-novel-cost-effective-approach-to-flow-visualization-using-parti
Publication Authors:
- Name: Raj, R.


In [33]:
# We will open and write on the csv file
Crawler_publication = 'CGL_crawled_data_publication.csv'
with open(Crawler_publication, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write header of each column
    writer.writerow(['Dates', 'Title', 'Publication_urls', 'Author', 'Author_Link']) 
    # For loop to correctly write the data.
    for i in range(len(website_data["Title"])):
        Dates = website_data["Dates"][i]
        Title = website_data["Title"][i]
        Publication_urls = website_data["Publication_urls"][i]
        Publication_authors = website_data["Publication_authors"][i]
        Author_Name = []
        Author_Link = []
        # for some publication there are more then one authur 
        for author in Publication_authors:
            Author_Name.append(author["name"])
            Author_Link.append(author["url"])
        # to write the values within the tables.
        writer.writerow([Dates, Title, Publication_urls, ', '.join(Author_Name), '\n'.join(Author_Link)])

In [35]:
# Count the number of titles
num_titles = len(website_data["Title"])
print("Number of Publications:", num_titles)

# Count the number of unique authors
unique_authors = set(author["name"] for publication_authors in website_data["Publication_authors"] for author in publication_authors)
num_unique_authors = len(unique_authors)
print("Number of Authors:", num_unique_authors)

Number of Publications: 478
Number of Authors: 66


#### DATA PREPROCESSING 

#### Step 1 is Normalization

In [39]:
# Initialize empty lists to store the extracted data
titles = []
author_names = []
author_links = []

# Loop through the website_data and extract the required information
for date, title, publication_url, publication_authors in zip(
        website_data["Dates"], website_data["Title"],
        website_data["Publication_urls"], website_data["Publication_authors"]
):
    # Extract Title
    titles.append(title)

    # Extract Author Name and Link
    author_names_list = []
    author_links_list = []
    for author in publication_authors:
        author_names_list.append(author["name"])
        author_links_list.append(author["url"])

    author_names.append(", ".join(author_names_list))
    author_links.append("\n".join(author_links_list))

In [41]:
#In this process we are making all the text to lower
CGL_Normalization = [title.lower() for title in titles]
print(CGL_Normalization)

['aging following a zero-temperature quench in the d = 3 ising model', 'a new phase diagram for fluid invasion patterns as a function of pore‐scale heterogeneity, surface roughness, and wettability', 'a novel cost-effective approach to flow visualization using particle streak angles', 'an ultra-precise fast fourier transform', 'colloidal deposits from evaporating sessile droplets: coffee ring versus surface capture', 'design and optimisation of an intra-aortic shrouded rotor axial pump', 'emergence of dissipation and hysteresis from interactions among reversible, non-dissipative units: the case of fluid-fluid interfaces', 'emergence of dissipation and hysteresis from interactions among reversible, non-dissipative units: the case of fluid-fluid interfaces', 'epidemiology of acute lymphoblastic leukaemia in sardinia, italy: age, sex, and environmental correlates', 'experimental study of a helical acoustic streaming flow', 'extremization to fine tune physics informed neural networks for s

#### Step 2 is Removing Punctions 

In [44]:
# Remove punctuation and special characters
cgl_cleaned = [re.sub(r'[^\w\s]', '', title) for title in CGL_Normalization]
print(cgl_cleaned)

['aging following a zerotemperature quench in the d  3 ising model', 'a new phase diagram for fluid invasion patterns as a function of porescale heterogeneity surface roughness and wettability', 'a novel costeffective approach to flow visualization using particle streak angles', 'an ultraprecise fast fourier transform', 'colloidal deposits from evaporating sessile droplets coffee ring versus surface capture', 'design and optimisation of an intraaortic shrouded rotor axial pump', 'emergence of dissipation and hysteresis from interactions among reversible nondissipative units the case of fluidfluid interfaces', 'emergence of dissipation and hysteresis from interactions among reversible nondissipative units the case of fluidfluid interfaces', 'epidemiology of acute lymphoblastic leukaemia in sardinia italy age sex and environmental correlates', 'experimental study of a helical acoustic streaming flow', 'extremization to fine tune physics informed neural networks for solving boundary value

#### Step 3 is Tokenization

In [47]:
#calling the libary 
from nltk.tokenize import word_tokenize
# Tokenize to be done on all the titles
cgl_Tokenization = [word_tokenize(title) for title in cgl_cleaned]
print(cgl_Tokenization)

[['aging', 'following', 'a', 'zerotemperature', 'quench', 'in', 'the', 'd', '3', 'ising', 'model'], ['a', 'new', 'phase', 'diagram', 'for', 'fluid', 'invasion', 'patterns', 'as', 'a', 'function', 'of', 'porescale', 'heterogeneity', 'surface', 'roughness', 'and', 'wettability'], ['a', 'novel', 'costeffective', 'approach', 'to', 'flow', 'visualization', 'using', 'particle', 'streak', 'angles'], ['an', 'ultraprecise', 'fast', 'fourier', 'transform'], ['colloidal', 'deposits', 'from', 'evaporating', 'sessile', 'droplets', 'coffee', 'ring', 'versus', 'surface', 'capture'], ['design', 'and', 'optimisation', 'of', 'an', 'intraaortic', 'shrouded', 'rotor', 'axial', 'pump'], ['emergence', 'of', 'dissipation', 'and', 'hysteresis', 'from', 'interactions', 'among', 'reversible', 'nondissipative', 'units', 'the', 'case', 'of', 'fluidfluid', 'interfaces'], ['emergence', 'of', 'dissipation', 'and', 'hysteresis', 'from', 'interactions', 'among', 'reversible', 'nondissipative', 'units', 'the', 'case', 

#### Step 4 is Stopwords

In [50]:
# Calling the library 
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
cgl_stopword = [[word for word in title if word not in stop_words] for title in cgl_Tokenization]
print(cgl_stopword)

[['aging', 'following', 'zerotemperature', 'quench', '3', 'ising', 'model'], ['new', 'phase', 'diagram', 'fluid', 'invasion', 'patterns', 'function', 'porescale', 'heterogeneity', 'surface', 'roughness', 'wettability'], ['novel', 'costeffective', 'approach', 'flow', 'visualization', 'using', 'particle', 'streak', 'angles'], ['ultraprecise', 'fast', 'fourier', 'transform'], ['colloidal', 'deposits', 'evaporating', 'sessile', 'droplets', 'coffee', 'ring', 'versus', 'surface', 'capture'], ['design', 'optimisation', 'intraaortic', 'shrouded', 'rotor', 'axial', 'pump'], ['emergence', 'dissipation', 'hysteresis', 'interactions', 'among', 'reversible', 'nondissipative', 'units', 'case', 'fluidfluid', 'interfaces'], ['emergence', 'dissipation', 'hysteresis', 'interactions', 'among', 'reversible', 'nondissipative', 'units', 'case', 'fluidfluid', 'interfaces'], ['epidemiology', 'acute', 'lymphoblastic', 'leukaemia', 'sardinia', 'italy', 'age', 'sex', 'environmental', 'correlates'], ['experimenta

In [52]:
#Calling the library
from nltk.stem import PorterStemmer
#Stemming
stemmer = PorterStemmer()  
cgl_stemming = [[PorterStemmer().stem(word) for word in title if word not in stop_words] for title in cgl_stopword]
print(cgl_stemming)

[['age', 'follow', 'zerotemperatur', 'quench', '3', 'ise', 'model'], ['new', 'phase', 'diagram', 'fluid', 'invas', 'pattern', 'function', 'porescal', 'heterogen', 'surfac', 'rough', 'wettabl'], ['novel', 'costeffect', 'approach', 'flow', 'visual', 'use', 'particl', 'streak', 'angl'], ['ultraprecis', 'fast', 'fourier', 'transform'], ['colloid', 'deposit', 'evapor', 'sessil', 'droplet', 'coffe', 'ring', 'versu', 'surfac', 'captur'], ['design', 'optimis', 'intraaort', 'shroud', 'rotor', 'axial', 'pump'], ['emerg', 'dissip', 'hysteresi', 'interact', 'among', 'revers', 'nondissip', 'unit', 'case', 'fluidfluid', 'interfac'], ['emerg', 'dissip', 'hysteresi', 'interact', 'among', 'revers', 'nondissip', 'unit', 'case', 'fluidfluid', 'interfac'], ['epidemiolog', 'acut', 'lymphoblast', 'leukaemia', 'sardinia', 'itali', 'age', 'sex', 'environment', 'correl'], ['experiment', 'studi', 'helic', 'acoust', 'stream', 'flow'], ['extrem', 'fine', 'tune', 'physic', 'inform', 'neural', 'network', 'solv', 'b

#### Now we will do the preprocessing for Authur names,

In [55]:
# Convert to lowercase
author_lower = [title.lower() for title in author_names]
# Remove punctuation and special characters
author_cleaned = [re.sub(r'[^\w\s]', '', title) for title in author_lower]
# Tokenize 
author_token = [word_tokenize(title) for title in author_cleaned]
# remove stopwords
stop_words = set(stopwords.words('english'))
author_stop = [[word for word in title if word not in stop_words] for title in author_token]
# Stemming
author_stemming = [[PorterStemmer().stem(word) for word in title if word not in stop_words] for title in author_stop]
#Print the output 
print(author_stemming)

[['gessert'], ['holtzman', 'r'], ['raj', 'r', 'thiruthumm', 'pothérat'], ['henri'], ['chubynski', 'v'], ['oran', 'e', 'henri'], ['holtzman', 'r', 'chubynski', 'v'], ['holtzman', 'r', 'chubynski'], ['carter', 'j'], ['pothérat'], ['thiruthumm', 'kim', 'e'], ['holtzman', 'r'], [], ['kim', 'ej', 'f'], ['thiruthumm', 'kim', 'e'], ['kim', 'e', 'f'], ['potherat'], ['bogush', 'wiel', 'v', 'holtzman', 'r'], ['kim', 'e', 'j'], ['ishaq'], ['pringl', 'c', 'c'], ['saeibehrouzi', 'holtzman', 'r'], ['saeibehrouzi', 'holtzman', 'r'], ['saeibehrouzi', 'holtzman', 'r'], ['saeibehrouzi', 'holtzman', 'r'], ['gianfrani', 'j'], ['kim', 'e'], ['holtzman', 'r'], ['carter', 'j'], ['bandara', 'kim', 'ej'], [], ['kim', 'e'], ['pringl', 'c'], ['henri'], ['henri'], ['kim', 'e'], ['choong', 'h', 'j', 'kim', 'e', 'f'], ['wald'], ['henri'], [], ['turner', 'maddock', 'h', 'kim', 'e'], [], ['holtzman', 'r'], ['kim', 'e'], ['henri'], ['wald'], ['wald'], ['carter', 'j'], [], ['pried', 'j'], ['holtzman', 'r'], [], ['kumar

#### Storing as CSV file

In this code we will store all the preprocessed title along with the other values in a csv file 

In [59]:
# We will open and write on the csv file
Title_update = 'CGL_crawled_data_publication_title.csv'
with open(Title_update, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write header of the csv file.
    writer.writerow(['Dates', 'Title', 'Publication_urls', 'Author', 'Author_Link', 'Title_preprocessed'])  
    for i in range(len(website_data["Title"])):
        Dates = website_data["Dates"][i]
        Title = website_data["Title"][i]
        Title_preprocessed = cgl_stopword[i]  # processed title
        Publication_urls = website_data["Publication_urls"][i]
        Publication_authors = website_data["Publication_authors"][i]
        Author_Name = []
        Author_Link = []
        # for some publication there are more then one authur 
        for author in Publication_authors:
            Author_Name.append(author["name"])
            Author_Link.append(author["url"])
        # to write the values within the tables.
        writer.writerow([Dates, Title, Publication_urls, ', '.join(Author_Name), '\n'.join(Author_Link), Title_preprocessed])

In this code we will store all the preprocessed name along with the other values in a csv file

In [62]:
import csv
# Existing CSV file
existing_csv_file = "CGL_crawled_data_publication_title.csv"
# Reading and storing its data
authur_list = []
with open(existing_csv_file, mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    authur_list = list(reader)
# Adding new heading
for i, row in enumerate(authur_list):
    if i == 0:
        row.append('author_stemming')  #Column heading
    else:
        row.append(author_stemming[i-1])  # list in which the data is there
# updated and new CSV file
updated_csv_file = "CGL_crawled_data_publication_final.csv"
# Writing on new CSV file
with open(updated_csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(existing_csv_file)

#### Fully Working query processor component with Cosine Similarity Score


In [ ]:
import csv
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# File on which we are going to work on.
CGL_Final = 'CGL_crawled_data_publication.csv'
# Getting the user query
user_query = input("Enter your query: ")
# All the same steps we used for preprocessing.
# step 1 Tokenization the String
tokens = word_tokenize(user_query)
# step 2 removing stopwords
stop_words = set(stopwords.words('english'))
# Filter out the stopwords
filtered_tokens = [w for w in tokens if not w.lower() in stop_words]
# step 3 Stemming
stemmer = PorterStemmer()  
stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
# step 4 Normalization
normalized_tokens = [token.lower() for token in stemmed_tokens]
# step 5 Removing punctuations
punctuations = string.punctuation
no_punct_tokens = [token for token in normalized_tokens if token not in punctuations]
preprocessed_query = ' '.join(no_punct_tokens)
# Read CSV file
with open(CGL_Final, 'r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    csv_data = list(reader)
# Preprocess CSV Data
def preprocess_row(row):
    stemmer = PorterStemmer()
    preprocessed_row = ' '.join([stemmer.stem(token.lower()) for token in row])
    no_punct_row = ''.join([char for char in preprocessed_row if char not in string.punctuation])
    return no_punct_row
corpus = [preprocess_row(row) for row in csv_data]
# Calculating Tfid matrix
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)
query_vector = vectorizer.transform([preprocessed_query])
found_results = []
for i, row in enumerate(csv_data):
    # using cosine similarity to calculate relevance score.
    relevance_score = cosine_similarity(tfidf_matrix[i], query_vector)[0][0]
    if relevance_score > 0:
        found_results.append((row, relevance_score))
# printing the matching output if there is any.
if found_results:
    print("Search Results:")
    found_results.sort(key=lambda x: x[1], reverse=True)
    for result in found_results:
        row, relevance_score = result
        date, heading, publication_url, *author_data = row
        print("Date:", date)
        print("Heading:", heading)
        print("Publication URL:", publication_url)
        print("Publication Authors:")
        for i in range(0, len(author_data), 2):
            author_name = author_data[i]
            author_url = author_data[i + 1] if i + 1 < len(author_data) else ''
            print("- Name:", author_name)
            if author_url.strip():
                print("- URL:", author_url)
        print("Cosine Similarity Score:", relevance_score)
        print("\n")
else:
    print("No results found.")

#### Fully Working query processor component with relevence score

In [ ]:
import csv
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
def preprocess_query(user_query):
    # All the same steps we used for preprocessing.
    # step 1 Tokenization the String
    tokens = word_tokenize(user_query)
    # step 2 removing stopwords
    stop_words = set(stopwords.words('english'))
    # Filter out the stopwords
    filtered_tokens = [w for w in tokens if not w.lower() in stop_words]
    # step 3 Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    # step 4 Normalization
    normalized_tokens = [token.lower() for token in stemmed_tokens]
    # step 5 Removing punctuations
    punctuations = string.punctuation
    no_punct_tokens = [token for token in normalized_tokens if token not in punctuations]
    preprocessed_query = ' '.join(no_punct_tokens)
    return preprocessed_query
#In this function we will read the csv file
def read_csv_file(filename):
    with open(filename, 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        return list(reader)
# In this part number of relevance word is compared
def search_in_csv(filename, query_tokens):
    stemmer = PorterStemmer()
    found_results = []
    csv_data = read_csv_file(filename)
    for row in csv_data:
        preprocessed_row = preprocess_row(row, stemmer)
        relevance_score = calculate_relevance_score(query_tokens, preprocessed_row)
        if relevance_score > 0:
            found_results.append((row, relevance_score))
    return found_results
def preprocess_row(row, stemmer):
    preprocessed_row = ' '.join([stemmer.stem(token.lower()) for token in row])
    stemmed_row = ' '.join([stemmer.stem(token) for token in preprocessed_row.split()])
    no_punct_row = ''.join([char for char in stemmed_row if char not in string.punctuation])
    return no_punct_row
# In this part number of relevance words is calculated.
def calculate_relevance_score(query_tokens, row):
    relevance_score = 0
    for token in query_tokens:
        if token in row:
            relevance_score += 1
    return relevance_score
def get_user_query():
    return input("Enter your query: ")
def display_search_results(found_results):
    if found_results:
        print("Search Results:")
        found_results.sort(key=lambda x: x[1], reverse=True)
        for result in found_results:
            row, relevance_score = result
            date, heading, publication_url, *author_data = row
            print("Date:", date)
            print("Heading:", heading)
            print("Publication URL:", publication_url)
            print("Publication Authors:")
            for i in range(0, len(author_data), 2):
                author_name = author_data[i]
                author_url = author_data[i + 1] if i + 1 < len(author_data) else ''
                print("- Name:", author_name)
                if author_url.strip():
                    print("- URL:", author_url)
                    
            print("Relevance Score:", relevance_score)
            print("\n")
    else:
        print("No results found.")
if __name__ == "__main__":
    filename = 'CGL_crawled_data_publication.csv'
    user_query = get_user_query()
    preprocessed_query = preprocess_query(user_query)
    query_tokens = preprocessed_query.split()
    found_results = search_in_csv(filename, query_tokens)
    display_search_results(found_results)

#### Fully Working query processor component with pop-up window

In [ ]:
import tkinter as tk
import csv
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
def preprocess_query(user_query):
    # Tokenization the String
    tokens = word_tokenize(user_query)
    # Get the set of English stopwords
    stop_words = set(stopwords.words('english'))
    # Filter out the stopwords
    filtered_tokens = [w for w in tokens if not w.lower() in stop_words]    
    # Stemming
    stemmer = PorterStemmer()  
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    # Normalization
    normalized_tokens = [token.lower() for token in stemmed_tokens]
    # Remove punctuations
    punctuations = string.punctuation
    no_punct_tokens = [token for token in normalized_tokens if token not in punctuations]
    return ' '.join(no_punct_tokens)
def read_csv_file(filename):
    with open(filename, 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        return list(reader)
def search_in_csv(filename, query_tokens):
    stemmer = PorterStemmer()
    found_results = []
    csv_data = read_csv_file(filename)
    corpus = [preprocess_row(row, stemmer) for row in csv_data]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)
    query_text = ' '.join(query_tokens)
    query_vector = vectorizer.transform([query_text])
    for i, row in enumerate(csv_data):
        relevance_score = cosine_similarity(tfidf_matrix[i], query_vector)[0][0]
        if relevance_score > 0:
            found_results.append((row, relevance_score))
    return found_results
def preprocess_row(row, stemmer):
    preprocessed_row = ' '.join([stemmer.stem(token.lower()) for token in row])
    stemmed_row = ' '.join([stemmer.stem(token) for token in preprocessed_row.split()])
    no_punct_row = ''.join([char for char in stemmed_row if char not in string.punctuation])
    return no_punct_row
def display_search_results(found_results):
    # Destroy any existing result_window before creating a new one
    if hasattr(display_search_results, "result_window"):
        display_search_results.result_window.destroy()
    # Create a new window for displaying the search results
    result_window = tk.Toplevel()
    result_window.title("Search Results")
    result_window.geometry("500x500")  # Set the size of the results window
    if found_results:
        # Create a text widget to display the search results
        text_widget = tk.Text(result_window, wrap=tk.WORD)
        text_widget.pack(fill=tk.BOTH, expand=True)
        found_results.sort(key=lambda x: x[1], reverse=True)
        for result in found_results:
            row, relevance_score = result
            date, heading, publication_url, *author_info = row
            authors = [author_info[i:i+2] for i in range(0, len(author_info), 2)]
            text_widget.insert(tk.END, f"Date: {date}\n")
            text_widget.insert(tk.END, f"Heading: {heading}\n")
            text_widget.insert(tk.END, f"Publication URL: {publication_url}\n")
            text_widget.insert(tk.END, "Publication Authors:\n")
            for author_name, author_url in authors:
                text_widget.insert(tk.END, f"- Name: {author_name}\n")
                if author_url.strip():
                    text_widget.insert(tk.END, f"- URL: {author_url}\n")
            text_widget.insert(tk.END, f"Cosine similarity Score: {relevance_score}\n\n")
    else:
        # If no results found, display a message in the window
        label = tk.Label(result_window, text="No results found.")
        label.pack()
    # Save the result_window reference to the function
    display_search_results.result_window = result_window
if __name__ == "__main__":
    filename = 'CGL_crawled_data_publication.csv'


In [ ]:
# Create the main window
window = tk.Tk()
# window title
window.title("centre-global-learning Search Engine")
# fixing window size
window.geometry("500x500")
# # Make window size fixed sise
# window.resizable(False, False)
# Create a label widget
label = tk.Label(window, text="Enter your query:")
label.pack()
# Create an entry widget for the user to input the query
entry = tk.Entry(window)
entry.pack()
def search_and_display():
    user_query = entry.get()
    preprocessed_query = preprocess_query(user_query)
    query_tokens = preprocessed_query.split()
    found_results = search_in_csv(filename, query_tokens)
    # Display the search results in a separate window
    display_search_results(found_results)
# search button and fuction
search_button = tk.Button(window, text="Search", command=search_and_display)
search_button.pack()
# Start the main event loop
window.mainloop()